In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats
from statistics import mean
import yfinance as yf

#Importing the list of the stock of S&P 500,
In the Ideal world, we would need to pay for the access but now we will be using static version

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')

#Acquiring an API Token

In [ ]:
import pandas_datareader as pdr
import datetime

# !pip install pandas==1.3.3 pandas-datareader==0.9.0


In [ ]:
df = pdr.get_data_tiingo('AAPL', api_key='API_KEY')


/usr/local/lib/python3.10/dist-packages/pandas_datareader/tiingo.py:234: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  return pd.concat(dfs, self._concat_axis)


In [ ]:
# Define the date range
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 12, 31)
symbol = 'AAPL'
TIINGO_API_KEY = 'API_KEY'

# Fetch data
df = pdr.get_data_tiingo(symbol, api_key=TIINGO_API_KEY, start=start_date, end=end_date)


/usr/local/lib/python3.10/dist-packages/pandas_datareader/tiingo.py:234: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  return pd.concat(dfs, self._concat_axis)


In [ ]:
latest_price = df['close'][-1]


In [ ]:
latest_price

192.53

In [ ]:
import requests


# Define the API URL for Tiingo (historical data)
symbol = 'AAPL'
api_url = f'https://api.tiingo.com/tiingo/daily/{symbol}/prices?token={TIINGO_API_KEY}'

# Make the request and get the data
response = requests.get(api_url)
data = response.json()

In [ ]:
data

[{'adjClose': 242.65,
  'adjHigh': 242.76,
  'adjLow': 238.9,
  'adjOpen': 239.81,
  'adjVolume': 38861017,
  'close': 242.65,
  'date': '2024-12-03T00:00:00+00:00',
  'divCash': 0.0,
  'high': 242.76,
  'low': 238.9,
  'open': 239.81,
  'splitFactor': 1.0,
  'volume': 38861017}]

In [ ]:
# Extract the latest price (close price)
latest_price = df['close'].iloc[0]
# Calculate the approximate market cap (close price * volume)
market_cap = df['close'].iloc[0] * df['volume'].iloc[0]

In [ ]:
market_cap

14022532097.97

#Adding our Stocks Data to Panda df

In [ ]:
my_colums = ['Ticker', 'Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_colums)
final_dataframe

,Ticker,Price,Market Capitalization,Number of Shares to Buy


In [ ]:
#Appening datapoints to this panda's data frame.
final_dataframe.append(
    pd.Series(
        [
            symbol,
            latest_price,
            market_cap,
            'N/A'
        ],
        index = my_colums
    ),
    ignore_index = True
)

,Ticker,Price,Market Capitalization,Number of Shares to Buy
0,AAPL,125.07,1.402253e+10,N/A


#Looping through the Tickers in Our list of Stocks
Using the same logic that, used above to pull all dta for S&P500 stocks and store their data in the DataFrame using a for loop

In [ ]:
final_dataframe = pd.DataFrame(columns=my_colums)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://api.tiingo.com/tiingo/daily/{stock}/prices?token={TIINGO_API_KEY}'
    data = requests.get(api_url).json()

    # Ensure the data is not empty
    if len(data) > 0:
        latest_data = data[0]  # Access the first item in the list (latest data point)
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    stock,
                    latest_data['close'],
                    latest_data['close'] * latest_data['volume'],  # Market Cap approximation
                    'N/A'  # Placeholder for additional data
                ],
                index=my_colums
            ),
            ignore_index=True
        )



In [ ]:

final_dataframe

,Ticker,Price,Market Capitalization,Number of Shares to Buy
0,A,139.28,2.388719e+08,N/A
1,AAL,14.47,2.333864e+08,N/A
2,AAP,42.64,7.401016e+07,N/A
3,AAPL,242.65,9.429626e+09,N/A
4,ABBV,181.50,7.455209e+08,N/A


## Using Batch API Calls to improve Performance
Because HTTP req are typically one of the slowest components of a script

In [ ]:
# Function to divide symbols into chunks of 100 (or as needed)
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
from time import sleep

# Your API key and the stock symbols
symbols = stocks['Ticker']  # Assuming stocks['Ticker'] is a list of 500 stock tickers
my_columns = ['Symbol', 'Latest Price', 'Market Cap Approx.', 'Additional Info']
final_dataframe = pd.DataFrame(columns=my_columns)

# Loop over each symbol to fetch data
for symbol in symbols:
    api_url = f'https://api.tiingo.com/tiingo/daily/{symbol}/prices?token={TIINGO_API_KEY}'
    print(f"Fetching data for: {symbol}")  # Debug: Check which symbol is being requested
    response = requests.get(api_url)

    # API response
    if response.status_code == 200:
        data = response.json()
        if data:  # Ensure the response contains data
            latest_data = data[0]  # Access the first data point
            final_dataframe = final_dataframe.append(
                pd.Series([
                    symbol,
                    latest_data['close'],  # Latest price
                    latest_data['close'] * latest_data['volume'],
                    'N/A'
                ], index=my_columns),
                ignore_index=True
            )
        else:
            print(f"No data found for {symbol}")  # Handle missing data
    else:
        print(f"Error fetching data for {symbol}: {response.status_code} - {response.text}")

    # Add delay to avoid hitting API rate limits
    sleep(0.5)

# Display the final dataframe
final_dataframe


Fetching data for: A
Fetching data for: AAL
Fetching data for: AAP
Fetching data for: AAPL
Fetching data for: ABBV
Fetching data for: ABC
No data found for ABC
Fetching data for: ABMD
Fetching data for: ABT
Fetching data for: ACN
Fetching data for: ADBE
Fetching data for: ADI
Fetching data for: ADM
Fetching data for: ADP
Fetching data for: ADSK
Fetching data for: AEE
Fetching data for: AEP
Fetching data for: AES
Fetching data for: AFL
Fetching data for: AIG
Fetching data for: AIV
Fetching data for: AIZ
Fetching data for: AJG
Fetching data for: AKAM
Fetching data for: ALB
Fetching data for: ALGN
Fetching data for: ALK
Fetching data for: ALL
Fetching data for: ALLE
Fetching data for: ALXN
Fetching data for: AMAT
Fetching data for: AMCR
Fetching data for: AMD
Fetching data for: AME
Fetching data for: AMGN
Fetching data for: AMP
Fetching data for: AMT
Fetching data for: AMZN
Error fetching data for AMZN: 429 - {"detail":"Error: You have run over your hourly request allocation. Please upgra

KeyboardInterrupt: 

In [ ]:
 final_dataframe

,Symbol,Latest Price,Market Cap Approx.,Additional Info
0,A,139.28,2.388719e+08,N/A
1,AAL,14.47,2.333864e+08,N/A
2,AAP,42.64,7.401016e+07,N/A
3,AAPL,242.65,9.429626e+09,N/A
4,ABBV,181.50,7.455209e+08,N/A
5,ABMD,381.02,9.830316e+04,N/A
6,ABT,116.29,5.575973e+08,N/A
7,ACN,352.54,1.073478e+09,N/A
8,ADBE,516.26,1.094086e+09,N/A
9,ADI,221.54,5.185079e+08,N/A


# Calculation the number of Shares to Buy


In [ ]:
portfolio_size = input("Enter the value of your Portfolio: ")
try:
  val = float(portfolio_size)
except ValueError:
  print("That's not a number! \n Please try again: ")
  portfolio_size = input("Enter the value of your Portfolio: ")
  val = float(portfolio_size)


Enter the value of your Portfolio: 1000000


In [ ]:
position_size = val/len(final_dataframe.index)
number_of_apple_shares = position_size/500
math.floor(number_of_apple_shares)

57

In [ ]:
for i in range(0, len(final_dataframe.index)):
  final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe['Latest Price'][i])


In [ ]:
final_dataframe


,Symbol,Latest Price,Market Cap Approx.,Additional Info,Number of Shares to Buy
0,A,139.28,2.388719e+08,N/A,205.0
1,AAL,14.47,2.333864e+08,N/A,1974.0
2,AAP,42.64,7.401016e+07,N/A,670.0
3,AAPL,242.65,9.429626e+09,N/A,117.0
4,ABBV,181.50,7.455209e+08,N/A,157.0
5,ABMD,381.02,9.830316e+04,N/A,74.0
6,ABT,116.29,5.575973e+08,N/A,245.0
7,ACN,352.54,1.073478e+09,N/A,81.0
8,ADBE,516.26,1.094086e+09,N/A,55.0
9,ADI,221.54,5.185079e+08,N/A,128.0


# Formating Our Excel Output
xlsxwriter lib for Python to create nicely-formated Excel files

###Initializing Our XlsxWrite Object

In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, sheet_name = 'Recommended Trades', index = False)

###Creating the Formats, That we would need for our .xlsx file
Formates include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:


*   Strings format for tickers
*   $XX.XX-format for stock prices

*   $XX.XXX-format for market capitalization
*   Integer formt for the number of shares to purchase






In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)



###Applying the Formats to the Columns of Our .xlsx File


We can use the set_column method applied to the writer.sheets['Recommended Trades'] object to apply formats to specific columns of our spreadsheets.

Here's an example:

writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )

In [ ]:
#MANUAL WAY TO DO THIS:

# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

In [ ]:
# USING FOR LOOP
column_formats = {
                    'A': ['Ticker', string_format],
                    'B': ['Latest Price', dollar_format],
                    'C': ['Market Cap Approx.', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

#Saving Excel Output


In [ ]:
writer.save()

/usr/local/lib/python3.10/dist-packages/xlsxwriter/workbook.py:369: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
